In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install laserembeddings==1.1.2
#!python -m laserembeddings download-models

from tqdm.auto import tqdm
import re
tqdm.pandas()
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from laserembeddings import Laser

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import sys
import os
os.system('python -m laserembeddings download-models')

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883989 sha256=77740cdda207f69129430f579be19b2b4271b58f6e99b1f885e5e1703308cf3f
  Stored in directory: /root/.cache/pip/wheels/d1/ff/0e/e00ff1e22100702ac8b24e709551ae0fb29db9ffc843510a64
Successfully built sacremoses
  Attempting uninstall: sacremoses
    Found existing installation: sacremoses 0.0.53
    Uninstalling sacremoses-0.0.53:
      Successfully uninstalled sacremoses-0.0.53
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.0 requires protobuf==3.20.0, but you have protobuf 3.19.4 which is incompatible.

✅   Downloaded https://dl.fbaipublicfiles.com/laser/mod

In [2]:
ls /opt/conda/lib/python3.7/site-packages/laserembeddings/data

93langs.fcodes  93langs.fvocab  bilstm.93langs.2018-12-26.pt


In [3]:
en_quran_tafsir = pd.read_csv('../input/assets/Quran_English_with_Tafseer.csv')
print(len(en_quran_tafsir.Tafseer))
en_quran_tafsir.head(2)

6236


,Name,Surah,Ayat,Verse,Tafseer
0,The Opening,1,1,"In the name of Allah, the Beneficent, the Merc...",In the Name of God the Compassionate the Merciful
1,The Opening,1,2,"Praise be to Allah, Lord of the Worlds,",In the Name of God the name of a thing is that...


In [4]:
bn_quran_tafsir = pd.read_csv('../input/assets/ben_quran_with_tafsir.csv')
print(len(bn_quran_tafsir.Sahih_International))
bn_quran_tafsir.head(2)

6236


,text,ayat,আল_বায়ান,তাইসিরুল,মুজিবুর_রহমান,Sahih_International,tafsir_bayan_headers,tafsir_bayan_text,tafsir_zakariya_headers,tafsir_zakariya_text
0,بِسۡمِ اللّٰهِ الرَّحۡمٰنِ الرَّحِیۡمِ ﴿۱﴾\nب...,"surah 1, ayat 1",পরম করুণাময় অতি দয়ালু আল্লাহর নামে।,(আরম্ভ করছি) পরম করুণাময় অসীম দয়াময় আল্লাহর না...,"পরম করুণাময়, অসীম দয়ালু আল্লাহর নামে (শুরু করছ...","In the name of Allah, the Entirely Merciful, t...",(১) অনন্ত করুণাময় পরম দয়ালু আল্লাহর নামে (আরম্...,‘বিসমিল্লাহ’র পূর্বে ‘আক্বরাউ’ ‘আবদাউ’ অথবা ‘আ...,"১. রহমান, রহীম(১) আল্লাহর নামে।(২)","১. সাধারণত আয়াতের অনুবাদে বলা হয়ে থাকে, পরম ..."
1,اَلۡحَمۡدُ لِلّٰهِ رَبِّ الۡعٰلَمِیۡنَ ۙ﴿۲﴾\n...,"surah 1, ayat 2","সমস্ত প্রশংসা আল্লাহর জন্য, যিনি সৃষ্টিকুলের রব।",যাবতীয় প্রশংসা জগৎসমূহের প্রতিপালক আল্লাহরই জন...,"আল্লাহরই জন্য সমস্ত প্রশংসা, যিনি বিশ্বজগতের র...","[All] praise is [due] to Allah, Lord of the wo...",(২) সমস্ত প্রশংসা[1] সারা জাহানের প্রতিপালক আল...,"[1] الحَمد এর মধ্যে যে ال রয়েছে, তা استغراق (...","২. সকল হামদ’(১) আল্লাহর(২), যিনি সৃষ্টিকুলের(৩...",১. আরবী ভাষায় হামদ অর্থ নির্মল ও সম্ভ্রমপূর্ণ...


In [5]:
 en_quran_tafsir[en_quran_tafsir.Tafseer.duplicated(keep=False)]

,Name,Surah,Ayat,Verse,Tafseer
7,The Cow,2,1,Alif. Lam. Mim.,Alif lām mīm God knows best what He means by t...
1074,The Heights,7,121,"Crying: We believe in the Lord of the Worlds,",They said ‘We believe in the Lord of the Worlds
1858,Al-Hijr,15,57,"He said: And afterward what is your business, ...",He said ‘So what is your business O you who ha...
2932,The Poets,26,1,Ta. Sin. Mim.,Tā sīn mīm God knows best what He means by the...
2978,The Poets,26,47,"Crying: We believe in the Lord of the Worlds,",They said ‘We believe in the Lord of the Worlds
...,...,...,...,...,...
5658,The Emissaries,77,37,Woe unto the repudiators on that day!,Woe to the deniers on that day!
5661,The Emissaries,77,40,Woe unto the repudiators on that day!,Woe to the deniers on that day!
5666,The Emissaries,77,45,Woe unto the repudiators on that day!,Woe to the deniers on that day!
5668,The Emissaries,77,47,Woe unto the repudiators on that day!,Woe to the deniers on that day!


In [6]:
 bn_quran_tafsir[bn_quran_tafsir.tafsir_bayan_text.duplicated(keep=False)]

,text,ayat,আল_বায়ান,তাইসিরুল,মুজিবুর_রহমান,Sahih_International,tafsir_bayan_headers,tafsir_bayan_text,tafsir_zakariya_headers,tafsir_zakariya_text
15,یُخٰدِعُوۡنَ اللّٰهَ وَ الَّذِیۡنَ اٰمَنُوۡا ...,"surah 2, ayat 9",তারা আল্লাহকে এবং যারা ঈমান এনেছে তাদেরকে ধোঁক...,"তারা আল্লাহ ও মু’মিনদেরকে প্রতারিত করে, আসলে ত...",তারা আল্লাহ ও মু’মিনদের সঙ্গে ধোঁকাবাজী করে। প...,They [think to] deceive Allah and those who be...,৯। আল্লাহ এবং বিশ্বাসীগণকে তারা প্রতারিত করতে ...,-তাফসীরে আহসানুল বায়ান,৯. আল্লাহ এবং মুমিনদেরকে তারা প্রতারিত করে। বস...,NaN
17,وَ اِذَا قِیۡلَ لَهُمۡ لَا تُفۡسِدُوۡا فِی ال...,"surah 2, ayat 11","আর যখন তাদেরকে বলা হয়, ‘তোমরা যমীনে ফাসাদ করো ...","তাদেরকে যখন বলা হয়, ‘পৃথিবীতে ফাসাদ সৃষ্টি করো...",এবং যখন তাদেরকে বলা হয়ঃ তোমরা পৃথিবীতে অশান্তি...,"And when it is said to them, ""Do not cause cor...","১১। তাদেরকে যখন বলা হয়, ‘পৃথিবীতে অশান্তি সৃষ্...",-তাফসীরে আহসানুল বায়ান,"১১. আর যখন তাদেরকে বলা হয়, তোমরা যমীনে ফাসাদ ...","১. আবুল আলীয়া বলেন, ফাসাদ সৃষ্টি করো না অর্থা..."
24,صُمٌّۢ بُکۡمٌ عُمۡیٌ فَهُمۡ لَا یَرۡجِعُوۡنَ ...,"surah 2, ayat 18",তারা বধির-মূক-অন্ধ। তাই তারা ফিরে আসবে না।,"তারা বধির, মূক, অন্ধ; কাজেই তারা (হিদায়াতের দি...","তারা বধির, মূক, অন্ধ। অতএব তারা প্রত্যাবৃত্ত হ...","Deaf, dumb and blind - so they will not return...","১৮। তারা বধির, বোবা ও অন্ধ; সুতরাং তারা ফিরবে না।",-তাফসীরে আহসানুল বায়ান,"১৮. তারা বধির, বোবা, অন্ধ, কাজেই তারা ফিরে আসব...","১. ইবনে আব্বাস (রাঃ) বলেন, এর অর্থ, তারা হেদায..."
25,اَوۡ کَصَیِّبٍ مِّنَ السَّمَآءِ فِیۡهِ ظُلُم...,"surah 2, ayat 19","কিংবা আকাশের বর্ষণমুখর মেঘের ন্যায়, যাতে রয়েছে...","অথবা যেমন আকাশের বর্ষণমুখর মেঘ, যাতে আছে গাঢ় অ...","অথবা আকাশ হতে বারি বর্ষণের ন্যায় যাতে অন্ধকার,...",Or [it is] like a rainstorm from the sky withi...,"১৯। কিংবা যেমন আকাশের মুষলধারা বৃষ্টি, যাতে রয়...",-তাফসীরে আহসানুল বায়ান,"১৯. কিংবা আকাশ হতে মুষলধারে বৃষ্টির ন্যায়, যা...","১. হাদীসে এসেছে, রাসূলুল্লাহ সাল্লাল্লাহু আলাই..."
27,یٰۤاَیُّهَا النَّاسُ اعۡبُدُوۡا رَبَّکُمُ الّ...,"surah 2, ayat 21","হে মানুষ, তোমরা তোমাদের রবের ইবাদাত কর, যিনি স...",হে মানুষ! ‘তোমরা তোমাদের সেই প্রতিপালকের ইবাদা...,হে মানববৃন্দ! তোমরা তোমাদের রবের ইবাদাত কর যিন...,"O mankind, worship your Lord, who created you ...",২১। হে মানুষ সম্প্রদায়! তোমরা তোমাদের সেই প্রত...,-তাফসীরে আহসানুল বায়ান,২১. হে মানুষ(১)! তোমরা তোমাদের সেই রব-এর(২) ইব...,১. আয়াতে উল্লেখিত ‘নাস’ আরবী ভাষায় সাধারণভাব...
...,...,...,...,...,...,...,...,...,...,...
6210,وَ لَاۤ اَنَا عَابِدٌ مَّا عَبَدۡتُّمۡ ۙ﴿۴﴾\n...,"surah 109, ayat 4",‘আর তোমরা যার ‘ইবাদত করছ আমি তার ‘ইবাদাতকারী হ...,আর আমি তার ‘ইবাদাতকারী নই তোমরা যার ‘ইবাদাত কর...,এবং আমি ইবাদাতকারী নই তার যার ইবাদাত তোমরা করে...,Nor will I be a worshipper of what you worship.,"৪। এবং আমি ইবাদতকারী নই তার, যার ইবাদত তোমরা ক...",-তাফসীরে আহসানুল বায়ান,৪. এবং আমি ইবাদতকারী নই তার যার ইবাদাত তোমরা ক...,-তাফসীরে জাকারিয়া
6213,اِذَا جَآءَ نَصۡرُ اللّٰهِ وَ الۡفَتۡحُ ۙ﴿۱﴾...,"surah 110, ayat 1","যখন আল্লাহর সাহায্য ও বিজয় আসবে,",যখন আসবে আল্লাহর সাহায্য ও (ইসলামের চূড়ান্ত) ব...,"যখন আসবে আল্লাহর সাহায্য ও বিজয়,",When the victory of Allah has come and the con...,১। যখন আসবে আল্লাহর সাহায্য ও বিজয়।,-তাফসীরে আহসানুল বায়ান,১. যখন আসবে আল্লাহর সাহায্য ও বিজয়(১),"(১) এ ব্যাপারে প্রায় সকলেই একমত যে, এখানে বিজ..."
6218,سَیَصۡلٰی نَارًا ذَاتَ لَهَبٍ ۚ﴿ۖ۳﴾\nسیصلی نا...,"surah 111, ayat 3",অচিরেই সে দগ্ধ হবে লেলিহান আগুনে।,"অচিরে সে প্রবেশ করবে লেলিহান শিখাযুক্ত আগুনে,",অচিরেই সে শিখা বিশিষ্ট জাহান্নামের আগুনে প্রবে...,He will [enter to] burn in a Fire of [blazing]...,৩। অচিরেই সে শিখাবিশিষ্ট (জাহান্নামের) আগুনে প...,-তাফসীরে আহসানুল বায়ান,"৩. অচিরে সে দগ্ধ হবে লেলিহান আগুনে(১),",(১) অর্থাৎ কেয়ামতে অথবা মৃত্যুর পর কবরেই সে এ...
6221,قُلۡ هُوَ اللّٰهُ اَحَدٌ ۚ﴿۱﴾\nقل هو الله احد...,"surah 112, ayat 1","বল, তিনিই আল্লাহ, এক-অদ্বিতীয়।","বল, তিনি আল্লাহ, এক অদ্বিতীয়,","বলঃ তিনিই আল্লাহ, একক/অদ্বিতীয়।","Say, ""He is Allah, [who is] One,","১। বল, তিনিই আল্লাহ একক (অদ্বিতীয়)।",-তাফসীরে আহসানুল বায়ান,"১. বলুন(১), তিনি আল্লাহ, এক-অদ্বিতীয়(২),","(১) বিভিন্ন বর্ণনায় এসেছে যে, মুশরিকরা রাসূলু

In [7]:
 bn_quran_tafsir[bn_quran_tafsir.tafsir_zakariya_text.duplicated(keep=False)]

,text,ayat,আল_বায়ান,তাইসিরুল,মুজিবুর_রহমান,Sahih_International,tafsir_bayan_headers,tafsir_bayan_text,tafsir_zakariya_headers,tafsir_zakariya_text
4,اِیَّاکَ نَعۡبُدُ وَ اِیَّاکَ نَسۡتَعِیۡنُ ؕ﴿...,"surah 1, ayat 5",আপনারই আমরা ইবাদাত করি এবং আপনারই নিকট আমরা সা...,আমরা কেবল তোমারই ‘ইবাদাত করি এবং কেবলমাত্র তোম...,আমরা আপনারই ইবাদাত করছি এবং আপনারই নিকট সাহায্...,It is You we worship and You we ask for help.,(৫) আমরা কেবল তোমারই ইবাদত করি এবং তোমারই কাছে...,"ইবাদতের অর্থ হল, কারো সন্তুষ্টি লাভের জন্য অত...",৫. আমরা শুধু আপনারই ইবাদাত(১) করি(২) এবং শুধু ...,NaN
8,ذٰلِکَ الۡکِتٰبُ لَا رَیۡبَ ۚۖۛ فِیۡهِ ۚۛ هُد...,"surah 2, ayat 2","এই সেই কিতাব, যাতে কোন সন্দেহ নেই, মুত্তাকীদের...","এটা ঐ (মহান) কিতাব যাতে কোন সন্দেহ নেই, মুত্তা...",ইহা ঐ গ্রন্থ যার মধ্যে কোন সন্দেহ-সংশয়ের অবকাশ...,This is the Book about which there is no doubt...,"২। এ গ্রন্থ; (কুরআন) এতে কোন সন্দেহ নেই, [1] স...",[1] এ কিতাবের অবতরণ যে আল্লাহর নিকট থেকে এ ব্য...,"২. এটা(১) সে কিতাব; যাতে কোন সন্দেহ নেই(২), মু...",NaN
15,یُخٰدِعُوۡنَ اللّٰهَ وَ الَّذِیۡنَ اٰمَنُوۡا ...,"surah 2, ayat 9",তারা আল্লাহকে এবং যারা ঈমান এনেছে তাদেরকে ধোঁক...,"তারা আল্লাহ ও মু’মিনদেরকে প্রতারিত করে, আসলে ত...",তারা আল্লাহ ও মু’মিনদের সঙ্গে ধোঁকাবাজী করে। প...,They [think to] deceive Allah and those who be...,৯। আল্লাহ এবং বিশ্বাসীগণকে তারা প্রতারিত করতে ...,-তাফসীরে আহসানুল বায়ান,৯. আল্লাহ এবং মুমিনদেরকে তারা প্রতারিত করে। বস...,NaN
34,کَیۡفَ تَکۡفُرُوۡنَ بِاللّٰهِ وَ کُنۡتُمۡ اَم...,"surah 2, ayat 28",কীভাবে তোমরা আল্লাহর সাথে কুফরী করছ অথচ তোমরা ...,তোমরা আল্লাহকে কীভাবে অস্বীকার করছ? অথচ তোমরা ...,কিরূপে তোমরা আল্লাহকে অবিশ্বাস করছ? অথচ তোমরা ...,How can you disbelieve in Allah when you were ...,২৮। তোমরা কিরূপে আল্লাহকে অস্বীকার কর? অথচ তোম...,(1) উক্ত আয়াতে দু’টি মরণ ও দু’টি জীবনের কথা উল...,২৮. তোমরা কিভাবে আল্লাহর সাথে কুফরী করছ? অথচ ত...,-তাফসীরে জাকারিয়া
38,قَالُوۡا سُبۡحٰنَکَ لَا عِلۡمَ لَنَاۤ اِلَّا ...,"surah 2, ayat 32","তারা বলল, ‘আপনি পবিত্র মহান। আপনি আমাদেরকে যা ...","তারা বলল, ‘আপনি পবিত্র মহান, আপনি আমাদেরকে যা ...",তারা বলেছিলঃ আপনি পরম পবিত্র! আপনি আমাদেরকে যা...,"They said, ""Exalted are You; we have no knowle...","৩২। তারা বলল, ‘আপনি মহান পবিত্র। আপনি আমাদেরকে...",-তাফসীরে আহসানুল বায়ান,"৩২. তারা বলল, আপনি পবিত্র মহান! আপনি আমাদেরকে ...",-তাফসীরে জাকারিয়া
...,...,...,...,...,...,...,...,...,...,...
6211,وَ لَاۤ اَنۡتُمۡ عٰبِدُوۡنَ مَاۤ اَعۡبُدُ ؕ﴿۵...,"surah 109, ayat 5",‘আর আমি যার ‘ইবাদাত করি তোমরা তার ‘ইবাদাতকারী ...,"আর আমি যার ‘ইবাদাত করি তোমরা তার ‘ইবাদাতকারী নও,",এবং তোমরা তাঁর ইবাদাতকারী নও যাঁর ইবাদাত আমি ক...,Nor will you be worshippers of what I worship.,"৫। এবং তোমরা তাঁর ইবাদতকারী নও, যাঁর ইবাদত আমি...",[1] কিছু মুফাসসির প্রথম আয়াতের অর্থকে বর্তমান ...,৫. এবং তোমরাও তার ইবাদতকারী হবে না যাঁর ইবাদত ...,-তাফসীরে জাকারিয়া
6212,لَکُمۡ دِیۡنُکُمۡ وَلِیَ دِیۡنِ ﴿۶﴾\nلکم دینک...,"surah 109, ayat 6",‘তোমাদের জন্য তোমাদের দীন আর আমার জন্য আমার দী...,তোমাদের পথ ও পন্থা তোমাদের জন্য (সে পথে চলার প...,তোমাদের জন্য তোমাদের কর্মফল এবং আমার জন্য আমার...,"For you is your religion, and for me is my rel...",৬। তোমাদের দ্বীন (শিরক) তোমাদের জন্য এবং আমার ...,"[1] অর্থাৎ, যদি তোমরা তোমাদের দ্বীন নিয়ে সন্তু...","৬. তোমাদের দ্বীন তোমাদের, আর আমার দ্বীন আমার।(১)",NaN
6230,قُلۡ اَعُوۡذُ بِرَبِّ النَّاسِ ۙ﴿۱﴾\nقل اعوذ ...,"surah 114, ayat 1","বল, ‘আমি আশ্রয় চাই মানুষের রব,","বল, ‘আমি আশ্রয় চাচ্ছি মানুষের প্রতিপালকের,","বলঃ আমি আশ্রয় চাচ্ছি মানুষের রবের,","Say, ""I seek refuge in the Lord of mankind,","১। বল, আমি আশ্রয় প্রার্থনা করছি মানুষের প্রতিপ...","[1] رَبّ (প্রতিপালক) এর অর্থ হল যে, যিনি শুরু ...","১. বলুন, আমি আশ্রয় প্রার্থনা করছি মানুষের রবের,",-তাফসীরে জাকারিয়া
6231,مَلِکِ النَّاسِ ۙ﴿۲﴾\nملک الناس ۙ﴿۲﴾,"surah 114, ayat 2","মানুষের অধিপতি,","মানুষের অধিপতির,",যিনি মানবমন্ডলীর বাদশাহ বা অধিপতি।,The Sovereign of mankind.,২। যিনি মানুষের মালিক। [1],[1] যে সত্তা সমস্ত মানুষের প্রতিপালন ও তত্ত্বা...,"২. মানুষের অধিপতির,",-তাফসীরে জাকারিয়া


from above 3 code cells it looks like the english tafsir version is better as it has only 84 duplicates where bangla version has many duplicates and missing samples (including empty tafsirs)

In [8]:
corpus_emb_quran = np.load('../input/assets/Holy_Quran_mlt_emb.npy')
corpus_emb_hadith = np.load('../input/assets/en_emb_bukhari_muslim.npy')

In [9]:
bn_quran_tafsir.আল_বায়ান[6]
en_bn_quran_tafsir = pd.read_csv('../input/assets/bn_translated_tafseer.csv')

In [10]:
bn_quran_tafsir['tafsir_bayan'] = bn_quran_tafsir['tafsir_bayan_headers'] + bn_quran_tafsir['tafsir_bayan_text']

In [11]:
%%time

def fix_translation_errors(bn_text):
    #solving hoshonto problem
    return re.sub(' ্ ','',bn_text)
    

en_quran_tafsir['ar_text'] = bn_quran_tafsir['text']
en_quran_tafsir['আল_বায়ান'] = bn_quran_tafsir['আল_বায়ান']
en_quran_tafsir['tafsir_bayan'] = bn_quran_tafsir['tafsir_bayan']
en_quran_tafsir['Bangla_Tafseer']=en_bn_quran_tafsir.Bangla_Tafseer.progress_apply(lambda bn_text: fix_translation_errors(bn_text))

en_quran_tafsir.head(1)

  0%|          | 0/6236 [00:00<?, ?it/s]

CPU times: user 71.4 ms, sys: 5.97 ms, total: 77.4 ms
Wall time: 76.6 ms


,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan,Bangla_Tafseer
0,The Opening,1,1,"In the name of Allah, the Beneficent, the Merc...",In the Name of God the Compassionate the Merciful,بِسۡمِ اللّٰهِ الرَّحۡمٰنِ الرَّحِیۡمِ ﴿۱﴾\nب...,পরম করুণাময় অতি দয়ালু আল্লাহর নামে।,(১) অনন্ত করুণাময় পরম দয়ালু আল্লাহর নামে (আরম্...,"পরম করুণাময়, পরম দয়ালু আল্লাহর নামে"


In [12]:
en_quran_tafsir.Bangla_Tafseer[6]

'আর তাদের পথ যাদেরকে তুমি আল্লাদীনার পথনির্দেশ দিয়ে অনুগ্রহ করেছ, তাদের পথের পরিবর্তে গহরিল-মাগধূবি (গহরিল-মাগধূবি) রয়েছে। ইহুদী ও খ্রিস্টানদের পথ নয়, যাদের উপর তোমার ক্রোধ রয়েছে। এই প্রতিস্থাপন দ্বারা বোঝানো সুক্ষ্ম অর্থ হচ্ছে যে, পথপ্রদর্শিত ব্যক্তিরা ইহুদী বা খ্রিস্টান নয়। আর আললাহজানেন যা সৎকরমপরায়ণতা , আর তাঁরই কাছে পরতযাবরতন ও পরতযাবরতন । আল্লাহ আমাদের মহাশয় মুহাম্মদ (সা.) এর পরিবার ও সহকর্মীদের প্রতি দোয়া কবুল করুন এবং তাদেরকে চিরন্তন শান্তি দান করুন। আমাদের জনয আললাহই যথেষট । তিনিই সরবশরেষঠ বনধু । কোনো শক্তি নেই , না কোনো শক্তি , কিনতু আললাহ-- তিনিই , মহাশকতিশালী , পরাকরমশালী ।'

In [13]:
en_quran_tafsir = en_quran_tafsir.drop(['Bangla_Tafseer'], axis=1)
en_quran_tafsir.to_csv('en_bn_quran_tafsir.csv',index = False)
en_quran_tafsir.head(1)

,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan
0,The Opening,1,1,"In the name of Allah, the Beneficent, the Merc...",In the Name of God the Compassionate the Merciful,بِسۡمِ اللّٰهِ الرَّحۡمٰنِ الرَّحِیۡمِ ﴿۱﴾\nب...,পরম করুণাময় অতি দয়ালু আল্লাহর নামে।,(১) অনন্ত করুণাময় পরম দয়ালু আল্লাহর নামে (আরম্...


In [14]:
def similarity(v1, v2):
    n1 = np.linalg.norm(v1)
    n2 = np.linalg.norm(v2)
    return np.dot(v1, v2) / n1 / n2

def pairwise_euclidean_dists(x, y):
    dists = -2 * np.matmul(x, y.T)
    dists +=  np.sum(x**2, axis=1)[:, np.newaxis]
    dists += np.sum(y**2, axis=1)
    return  np.sqrt(dists)


laser = Laser()

In [15]:
en_bn_translated_sahih_hadiths =  pd.read_csv('../input/assets/en_bn_translated_sahih_hadiths.csv')
en_bn_translated_sahih_hadiths.head(1)

,id,hadith_id,source,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en,narrators,text_bn
0,0,1,Sahih Bukhari,1,1,Revelation - كتاب بدء الوحى,"30418, 20005, 11062, 11213, 11042, 3",حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...,Narrated 'Umar bin Al-Khattab: ...,'Abdullah bin al-Zubair bin 'Isa عبد الله بن ا...,উমর বিন আল-খাততাবের কাহিনীঃ আমি আল্লার রসূলকে ...


In [16]:
en_bukhari_muslim = pd.read_csv('../input/assets/en_bukhari_muslim.csv')
en_bukhari_muslim['text_bn'] = en_bn_translated_sahih_hadiths['text_bn']
en_bn_bukhari_muslim = en_bukhari_muslim.drop(['chain_indx','id','hadith_id'], axis=1)
en_bn_bukhari_muslim.head(1)

,source,chapter_no,hadith_no,chapter,text_ar,text_en,narrators,text_bn
0,Sahih Bukhari,1,1,Revelation,حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...,Narrated 'Umar bin Al-Khattab: ...,"'Abdullah bin al Zubair bin 'Isa ,Sufyan bin '...",উমর বিন আল-খাততাবের কাহিনীঃ আমি আল্লার রসূলকে ...


In [17]:

en_bn_bukhari_muslim.to_csv('en_bn_translated_sahih_hadiths_filtered.csv',index = False)

In [18]:
def MLT_Sahih_Hadith_Search_Engine(query,size=10,language = 'en',metric = 'dot',query_embedding=None):

    if(metric == 'dot'):
        query_embedding = np.squeeze(np.asarray(query_embedding))
        linear_similarities = similarity(corpus_emb_hadith, query_embedding)
    else:
        linear_similarities = pairwise_euclidean_dists(corpus_emb_hadith, query_embedding)
        linear_similarities = np.squeeze(np.asarray(linear_similarities))
        linear_similarities = np.array(linear_similarities, dtype=np.float32)

    if(metric == 'dot'):
        Top_index_doc = linear_similarities.argsort()[:-(size+1):-1]
    else:
        Top_index_doc = linear_similarities.argsort()[:-(size+1):]

    linear_similarities.sort()
    find = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        find.loc[i,'source'] = str(en_bn_bukhari_muslim['source'][index])
        find.loc[i,'chapter_no'] = str(en_bn_bukhari_muslim['chapter_no'][index])
        find.loc[i,'hadith_no'] = str(en_bn_bukhari_muslim['hadith_no'][index])
        find.loc[i,'chapter'] = str(en_bn_bukhari_muslim['chapter'][index])
        find.loc[i,'text_ar'] = str(en_bn_bukhari_muslim['text_ar'][index])
        find.loc[i,'text_en'] = str(en_bn_bukhari_muslim['text_en'][index])
        find.loc[i,'text_bn'] = str(en_bn_bukhari_muslim['text_bn'][index])
        find.loc[i,'narrators'] = str(en_bn_bukhari_muslim['narrators'][index])
        
    if(metric == 'dot'):
        for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
            find.loc[j,'Score'] = simScore
    else:
        for j,simScore in enumerate(linear_similarities[:-(size+1):]):
            find.loc[j,'Score'] = simScore
        
    return find

In [19]:
def Multilingual_Quran_Hadith_Search_Engine(query,size=10,language = 'en',metric = 'dot'):

    query_embedding = laser.embed_sentences(query, lang=language)

    mlt_hadiths = MLT_Sahih_Hadith_Search_Engine(query,size=3,language = 'en',metric = 'dot',query_embedding=query_embedding)
    
    if(metric == 'dot'):
        query_embedding = np.squeeze(np.asarray(query_embedding))
        linear_similarities = similarity(corpus_emb_quran, query_embedding)
    else:
        linear_similarities = pairwise_euclidean_dists(corpus_emb_quran, query_embedding)
        linear_similarities = np.squeeze(np.asarray(linear_similarities))
        linear_similarities = np.array(linear_similarities, dtype=np.float32)

    if(metric == 'dot'):
        Top_index_doc = linear_similarities.argsort()[:-(size+1):-1]
    else:
        Top_index_doc = linear_similarities.argsort()[:-(size+1):]

    linear_similarities.sort()
    find = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        find.loc[i,'Name'] = str(en_quran_tafsir['Name'][index])
        find.loc[i,'Surah'] = str(en_quran_tafsir['Surah'][index])
        find.loc[i,'Ayat'] = str(en_quran_tafsir['Ayat'][index])
        find.loc[i,'Verse'] = en_quran_tafsir['Verse'][index] 
        find.loc[i,'Tafseer'] = en_quran_tafsir['Tafseer'][index] 
        
    if(metric == 'dot'):
        for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
            find.loc[j,'Score'] = simScore
    else:
        for j,simScore in enumerate(linear_similarities[:-(size+1):]):
            find.loc[j,'Score'] = simScore
        
    bn_find = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        bn_find.loc[i,'arabic_text'] = str(bn_quran_tafsir['text'][index])
        bn_find.loc[i,'Surah_ayat'] = str(bn_quran_tafsir['ayat'][index])
        bn_find.loc[i,'আল_বায়ান'] = str(bn_quran_tafsir['আল_বায়ান'][index])
        bn_find.loc[i,'tafsir_bayan'] = bn_quran_tafsir['tafsir_bayan_text'][index] 
        bn_find.loc[i,'tafsir_zakariya'] = bn_quran_tafsir['tafsir_zakariya_text'][index] 
        
    if(metric == 'dot'):
        for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
            bn_find.loc[j,'Score'] = simScore
    else:
        for j,simScore in enumerate(linear_similarities[:-(size+1):]):
            bn_find.loc[j,'Score'] = simScore
    if size==1:
        print("Query: ",query,".  Surah:", find.loc[0,'Name'], " Number:", find.loc[0,'Surah'], 
              " Ayat:", find.loc[0,'Ayat']," Tafseer: ", find.loc[0,'Tafseer'], "\n")
    else:
        return find,bn_find,mlt_hadiths

In [20]:
%%time
find,bn_find,mlt_hadiths = Multilingual_Quran_Hadith_Search_Engine("এই হচ্ছে মারইয়াম পুত্র ঈসা। এটাই সঠিক বক্তব্য",size=2,language = 'en')

CPU times: user 687 ms, sys: 151 ms, total: 838 ms
Wall time: 543 ms


In [21]:
find

,Name,Surah,Ayat,Verse,Tafseer,Score
0,Mary,19,34,"Such was Jesus, son of Mary: (this is) a state...",That is Jesus son of Mary a statement of truth...,0.012600
1,The Believers,23,50,And We made the son of Mary and his mother a p...,And We made the son of Mary Jesus and his moth...,0.010997


In [22]:
bn_find

,arabic_text,Surah_ayat,আল_বায়ান,tafsir_bayan,tafsir_zakariya,Score
0,ذٰلِکَ عِیۡسَی ابۡنُ مَرۡیَمَ ۚ قَوۡلَ الۡحَق...,"surah 19, ayat 34","এই হচ্ছে মারইয়াম পুত্র ঈসা। এটাই সঠিক বক্তব্য,...",[1] এ হল সেই সকল গুণাবলী যা ঈসা (আঃ)-এর মধ্যে ...,(১) ঈসা আলাইহিস সালাম সম্পর্কে নাসারারা তাঁর প...,0.012600
1,وَ جَعَلۡنَا ابۡنَ مَرۡیَمَ وَ اُمَّهٗۤ اٰیَۃ...,"surah 23, ayat 50",আর আমি মারইয়াম-পুত্র ও তার মাকে নিদর্শন বানালা...,[1] কারণ ঈসা (আঃ) এর জন্ম হয়েছিল বিনা পিতায় যা...,(১) আভিধানিক অর্থে “রাবওয়াহ” এমন সুউচ্চ ভূমিক...,0.010997


In [23]:
mlt_hadiths

,source,chapter_no,hadith_no,chapter,text_ar,text_en,text_bn,narrators,Score
0,Sahih Bukhari,64,4193,Military Expeditions led by the Prophet pbuh A...,حدثنا عبد العزيز بن عبد الله، حدثنا إبراهيم بن...,Narrated `Aisha: Whenever...,"` আয়েশা (রাঃ) বলেন, যখনই আল্লাহর রসূল (সাঃ) ক...","'Abdul 'Aziz bin 'Abdullah al Uvaisi ,Ibrahim ...",0.004165
1,Sahih Bukhari,54,2732,Conditions,حدثني عبد الله بن محمد، حدثنا عبد الرزاق، أخبر...,Narrated Al-Miswar bin Makhrama and Marwan: ...,আল-মিসওয়ার বিন মখরমা ও মারওয়ান বর্ণনা করেছেন...,"'Abdullah bin Muhammad al Musandi ,'Abdur Razz...",0.003988
2,Sahih Bukhari,87,6918,Punishment of Disbelievers at War with Allah a...,حدثنا عبد العزيز بن عبد الله، حدثني إبراهيم بن...,Narrated Ibn `Abbas: I us...,ইবনে আব্বাস বলেনঃ আমি মুহাজিরদের একদলকে শিক্ষা...,"'Abdul 'Aziz bin 'Abdullah al Uvaisi ,Ibrahim ...",0.003921


In [24]:
find,bn_find,mlt_hadiths = Multilingual_Quran_Hadith_Search_Engine("আমি কি পুরস্কার পেতে পারি?",language = 'bn')
find

,Name,Surah,Ayat,Verse,Tafseer,Score
0,The Event,56,24,Reward for what they used to do.,a reward jazā’an is an object denoting reason ...,0.009955
1,The Reality,69,47,And not one of you could have held Us off from...,and not one of you ahadin is the subject of mā...,0.009949
2,The Cave,18,102,Do the disbelievers reckon that they can choos...,Do the disbelievers reckon that they can take ...,0.009504
3,The Prophets,21,111,And I know not but that this may be a trial fo...,I do not know; perhaps that which I have notif...,0.009492
4,The Prophets,21,21,Or have they chosen gods from the earth who ra...,Or am functions with the meaning of bal ‘nay’ ...,0.009294
5,He Frowned,80,4,Or take heed and so the reminder might avail h...,or be admonished yadhdhakkar the original tā’ ...,0.009285
6,He Frowned,80,13,On honoured leaves,on leaves fī suhufin is the second predicate o...,0.009268
7,Qâf,50,29,The sentence that cometh from Me cannot be cha...,The word that comes from Me in this respect ca...,0.009257
8,The Pen,68,9,"Who would have had thee compromise, that they ...",They desire they yearn that law relates to the...,0.009231
9,The Poets,26,14,"And they have a crime against me, so I fear th...",And I have sinned against them by my slaying o...,0.009218


In [25]:
bn_find

,arabic_text,Surah_ayat,আল_বায়ান,tafsir_bayan,tafsir_zakariya,Score
0,جَزَآءًۢ بِمَا کَانُوۡا یَعۡمَلُوۡنَ ﴿۲۴﴾\nج...,"surah 56, ayat 24",তারা যে আমল করত তার প্রতিদানস্বরূপ।,-তাফসীরে আহসানুল বায়ান,-তাফসীরে জাকারিয়া,0.009955
1,فَمَا مِنۡکُمۡ مِّنۡ اَحَدٍ عَنۡهُ حٰجِزِیۡنَ...,"surah 69, ayat 47",অতঃপর তোমাদের মধ্যে কেউই তাকে রক্ষা করার থাকত ...,"[1] এ থেকে জানা গেল যে, মুহাম্মাদ (সাঃ) সত্য র...",-তাফসীরে জাকারিয়া,0.009949
2,اَفَحَسِبَ الَّذِیۡنَ کَفَرُوۡۤا اَنۡ یَّتَّخ...,"surah 18, ayat 102","যারা কুফরী করছে, তারা কি মনে করেছে যে, তারা আম...","[1] حَسب অর্থ ধারণা করা, عِبادي আমার দাস বা বা...","(১) عِبَادِي (আমার দাস) বলে এখানে ফিরিশতা, নেক...",0.009504
3,وَ اِنۡ اَدۡرِیۡ لَعَلَّهٗ فِتۡنَۃٌ لَّکُمۡ و...,"surah 21, ayat 111",আর আমি জানি না হয়তো তা তোমাদের জন্য এক পরীক্ষা...,-তাফসীরে আহসানুল বায়ান,(১) অর্থাৎ এ বিলম্বের কারণে তোমরা পরীক্ষার মুখ...,0.009492
4,اَمِ اتَّخَذُوۡۤا اٰلِهَۃً مِّنَ الۡاَرۡضِ هُ...,"surah 21, ayat 21","তারা যেসব মাটির দেবতা গ্রহণ করেছে, সেগুলি কি ম...","[1] জিজ্ঞাসা অস্বীকৃতির জন্য। অর্থাৎ, তারা তা ...","(১) ‘ইনশার’ মানে হচ্ছে, কোন পড়ে থাকা প্রাণহীন...",0.009294
5,اَوۡ یَذَّکَّرُ فَتَنۡفَعَهُ الذِّکۡرٰی ؕ﴿۴﴾\...,"surah 80, ayat 4","অথবা উপদেশ গ্রহণ করত, ফলে সে উপদেশ তার উপকারে ...",-তাফসীরে আহসানুল বায়ান,(১) অর্থাৎ আপনি কি জানেন এই সাহাবী যা জিজ্ঞেস ...,0.009285
6,فِیۡ صُحُفٍ مُّکَرَّمَۃٍ ﴿ۙ۱۳﴾\nفی صحف مکرمۃ ...,"surah 80, ayat 13",এটা আছে সম্মানিত সহীফাসমূহে।*,"[1] অর্থাৎ, লওহে মাহ্ফূযে সংরক্ষিত আছে। কেননা,...",-তাফসীরে জাকারিয়া,0.009268
7,مَا یُبَدَّلُ الۡقَوۡلُ لَدَیَّ وَ مَاۤ اَنَا...,"surah 50, ayat 29","‘আমার কাছে কথা রদবদল হয় না, আর আমি বান্দার প্র...","[1] অর্থাৎ, যে সব অঙ্গীকার আমি করেছি, তার বিপর...",-তাফসীরে জাকারিয়া,0.009257
8,وَدُّوۡا لَوۡ تُدۡهِنُ فَیُدۡهِنُوۡنَ ﴿۹﴾\nود...,"surah 68, ayat 9","তারা কামনা করে, যদি তুমি আপোষকামী হও, তবে তারা...","[1] অর্থাৎ, তারা তো এটাই চায় যে, তুমি তাদের উপ...",-তাফসীরে জাকারিয়া,0.009231
9,وَ لَهُمۡ عَلَیَّ ذَنۡۢبٌ فَاَخَافُ اَنۡ یَّق...,"surah 26, ayat 14",‘আর আমার বিরুদ্ধে তাদের কাছে একটি অপরাধের অভিয...,"[1] এখানে ইঙ্গিত করা হয়েছে ঐ হত্যার প্রতি, যা ...",-তাফসীরে জাকারিয়া,0.009218


In [26]:
mlt_hadiths

,source,chapter_no,hadith_no,chapter,text_ar,text_en,text_bn,narrators,Score
0,Sahih Bukhari,87,6918,Punishment of Disbelievers at War with Allah a...,حدثنا عبد العزيز بن عبد الله، حدثني إبراهيم بن...,Narrated Ibn `Abbas: I us...,ইবনে আব্বাস বলেনঃ আমি মুহাজিরদের একদলকে শিক্ষা...,"'Abdul 'Aziz bin 'Abdullah al Uvaisi ,Ibrahim ...",0.007022
1,Sahih Muslim,51,7196,The Book Pertaining to Repentance and Exhortat...,حدثنا حبان بن موسى، أخبرنا عبد الله بن المبارك...,"Sa'id b. Musayyib, 'Urwa b. Zubair, 'Alqama b...","সাঈদ খ. মুসাইব, 'উরুওয়া খ. জুবাইর, 'আলকামা খ....","Hiban bin Musa bin Sawar ,'Abdullah bin Mubara...",0.006431
2,Sahih Bukhari,52,2661,Witnesses,حدثنا أبو الربيع، سليمان بن داود وأفهمني بعضه ...,Narrated Aisha: (the wife...,আয়েশা (আঃ) এর বৎসরী বর্ণনা: (নবী (সাঃ) এর স্ত...,"Sulaiman bin Da'ud al 'Atki al Zahrani ,Falayh...",0.006297


In [27]:
# !pip install faiss-cpu

# import faiss
# import numpy as np
# import typing as tp
# import os
# from enum import Enum


# class Margin(Enum):
#     RATIO = "ratio"
#     DISTANCE = "distance"
#     ABSOLUTE = "absolute"

#     @classmethod
#     def has_value(cls, value):
#         return value in cls._value2member_map_
# def _score_knn(x: np.ndarray, y: np.ndarray, k: int, margin: str) -> np.ndarray:
#     nbex, dim = x.shape
#     # create index
#     idx_x = faiss.IndexFlatIP(dim)
#     idx_y = faiss.IndexFlatIP(dim)
#     # L2 normalization needed for cosine distance
#     faiss.normalize_L2(x)
#     faiss.normalize_L2(y)
#     idx_x.add(x)
#     idx_y.add(y)
#     if margin == Margin.ABSOLUTE.value:
#         scores, indices = idx_y.search(x, 1)
#     else:
#         # return cosine similarity and indices of k closest neighbors
#         Cos_xy, Idx_xy = idx_y.search(x, k)
#         Cos_yx, Idx_yx = idx_x.search(y, k)

#         # average cosines
#         Avg_xy = Cos_xy.mean(axis=1)
#         Avg_yx = Cos_yx.mean(axis=1)

#         scores = _score_margin(Cos_xy, Idx_xy, Avg_xy, Avg_yx, margin, k)

#         # find best
#         best = scores.argmax(axis=1)
#         indices = np.zeros((nbex, 1), dtype=np.int32)
#         for i in range(nbex):
#             indices[i] = Idx_xy[i, best[i]]
#     return indices


# def _score_margin(
#     Dxy: np.ndarray,
#     Ixy: np.ndarray,
#     Ax: np.ndarray,
#     Ay: np.ndarray,
#     margin: str,
#     k: int,
# ) -> np.ndarray:
#     nbex = Dxy.shape[0]
#     scores = np.zeros((nbex, k))
#     for i in range(nbex):
#         for j in range(k):
#             jj = Ixy[i, j]
#             a = Dxy[i, j]
#             b = (Ax[i] + Ay[jj]) / 2
#             if margin == Margin.RATIO.value:
#                 scores[i, j] = a / b
#             else:  # distance margin
#                 scores[i, j] = a - b
#     return scores

# def calculate_error(
#     x: np.ndarray,
#     y: np.ndarray,
#     margin: str = None,
#     k: int = 4,
#     eval_text: str = None,
# ) -> tp.Tuple[int, int]:
#     assert (
#         x.shape == y.shape
#     ), f"number of source {x.shape} / target {y.shape} shapes mismatch"
#     nbex = x.shape[0]

#     # for each x calculate the highest scoring neighbor from y
#     closest_neighbor = _score_knn(x, y, k, margin)

#     if eval_text:  # calc textual error
#         lines = open(eval_text, encoding="utf-8", errors="surrogateescape").readlines()
#         err = 0
#         for ex in range(nbex):
#             if lines[ex] != lines[closest_neighbor[ex, 0]]:
#                 err += 1
#     else:  # calc index error
#         ref = np.linspace(0, nbex - 1, nbex).astype(int)  # [0, nbex)
#         err = nbex - np.equal(closest_neighbor.reshape(nbex), ref).astype(int).sum()
#     return err, nbex

In [28]:
# query_embedding = laser.embed_sentences("আমি কি পুরস্কার পেতে পারি?", lang='bn')

# #query_embedding = np.squeeze(np.asarray(query_embedding))
# #linear_similarities = similarity(corpus_emb_quran, query_embedding)
# # corpus_emb_quran[0].shape
# B = np.reshape(corpus_emb_quran[10], (-1, 1))
# B= B.T
# B.shape
# err, nbex = calculate_error(B , query_embedding, margin =  Margin.RATIO.value, k = 4, eval_text = False)
# nbex

In [29]:
import site
loc = site.getsitepackages() 
print(site.getusersitepackages()) # String for user-specific package location)
loc[0]

/root/.local/lib/python3.7/site-packages


'/opt/conda/lib/python3.7/site-packages'